все аналогично reading_xml_purchases.ipynb, только тут другие файлы

In [ ]:
!gdown 1r2u9Spe7_gK4VN24A8PYk2bROTIv8BSM
!gdown 1NnaF1CLsC5RiP-nB9S5ElsjzxiVcs9_f
!7z x zakupki.7z
!7z x contract.7z

Downloading...
From: https://drive.google.com/uc?id=1r2u9Spe7_gK4VN24A8PYk2bROTIv8BSM
To: /content/zakupki.7z
100% 1.30G/1.30G [00:10<00:00, 128MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NnaF1CLsC5RiP-nB9S5ElsjzxiVcs9_f
To: /content/contract.7z
100% 929M/929M [00:08<00:00, 105MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1304205494 bytes (1244 MiB)

Extracting archive: zakupki.7z
--
Path = zakupki.7z
Type = 7z
Physical Size = 1304205494
Headers Size = 67832
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 2        0% 18 - zakupki/contract_Sankt-Peterbur . 0190110_235959_daily_005.xml.zip                                                       

In [ ]:
!pip install lxml beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from zipfile import ZipFile, BadZipFile

from bs4 import BeautifulSoup

from collections import defaultdict
from tqdm import tqdm
import pandas as pd

In [ ]:
with ZipFile("/content/zakupki/contract_Sankt-Peterburg_20190109_000000_20190109_235959_daily_004.xml.zip", 'r') as zObject:
    zObject.extractall("/content/zakupki_out")

In [ ]:
filepath = '/content/zakupki_out/contract_Sankt-Peterburg_20190109_000000_20190109_235959_daily_003.xml'

with open(filepath, 'r') as f:
	data = f.read()

# 'xml' is the parser used. For html files, which BeautifulSoup is typically used for, it would be 'html.parser'.
soup = BeautifulSoup(data, 'xml')

In [ ]:
len(soup.find_all('ns2:item'))

1

In [ ]:
key_xml = {
    'guid': 'ns2:guid',
    'registrationNumber': 'ns2:registrationNumber',
    'notice44': 'ns2:notice44',
    'noticeNotPlacedByFz223p5s4': 'ns2:noticeNotPlacedByFz223p5s4',
    'notice44Num': 'ns2:notice44Num',
    'termination': 'ns2:termination',
    'urlEIS': 'ns2:urlEIS',
    'createDateTime': 'ns2:createDateTime',
    'publicationDate': 'ns2:publicationDate',
    'contractDate': 'ns2:contractDate',
    'subjectContract': 'ns2:subjectContract',
    'customer_fullName': ['ns2:customer', 'mainInfo', 'fullName'],
    'customer_shortName': ['ns2:customer', 'mainInfo', 'shortName'],
    'customer_inn': ['ns2:customer', 'mainInfo', 'inn'],
    'placer_fullName': ['ns2:placer', 'mainInfo', 'fullName'],
    'placer_shortName': ['ns2:placer', 'mainInfo', 'shortName'],
    'placer_inn': ['ns2:placer', 'mainInfo', 'inn'],
    'price': 'ns2:price',
    'currency_code': ['ns2:currency', 'code'],
    'startExecutionDate': 'ns2:startExecutionDate',
    'endExecutionDate': 'ns2:endExecutionDate',
}

In [ ]:
os.makedirs('zakupki_out', exist_ok=True)

for filename in tqdm(os.listdir('/content/zakupki')):
    try:
        with ZipFile(f"/content/zakupki/{filename}", 'r') as zObject:
            zObject.extractall("/content/zakupki_out")
    except BadZipFile as e:
        pass

100%|██████████| 6216/6216 [03:33<00:00, 29.13it/s]


In [ ]:
zakupki_data = defaultdict(list)

for fn in tqdm(os.listdir('/content/zakupki_out')):
    filepath = f"/content/zakupki_out/{fn}"

    with open(filepath, 'r') as f:
        data = f.read()

    if len(data) == 0:
        continue

    soup = BeautifulSoup(data, 'xml')

    for item in soup.find_all('ns2:item'):
        for k, path in key_xml.items():
            try:
                if isinstance(path, str):
                    zakupki_data[k].append(item.find(path).text)
                else:
                    item_ = item
                    for p in path:
                        item_ = item_.find(p)
                    zakupki_data[k].append(item_.text)
            except AttributeError:
                zakupki_data[k].append(None)

zakupki_df = pd.DataFrame(zakupki_data)

100%|██████████| 177820/177820 [1:09:14<00:00, 42.80it/s]


In [ ]:
os.makedirs('contract_out', exist_ok=True)

for filename in tqdm(os.listdir('/content/contract')):
    try:
        with ZipFile(f"/content/contract/{filename}", 'r') as zObject:
            zObject.extractall("/content/contract_out")
    except BadZipFile as e:
        pass

100%|██████████| 4361/4361 [02:10<00:00, 33.47it/s]


In [ ]:
contract_data = defaultdict(list)

for fn in tqdm(os.listdir('/content/contract_out')):
    filepath = f"/content/contract_out/{fn}"

    with open(filepath, 'r') as f:
        data = f.read()

    if len(data) == 0:
        continue

    soup = BeautifulSoup(data, 'xml')

    for item in soup.find_all('ns2:item'):
        for k, path in key_xml.items():
            try:
                if isinstance(path, str):
                    contract_data[k].append(item.find(path).text)
                else:
                    item_ = item
                    for p in path:
                        item_ = item_.find(p)
                    contract_data[k].append(item_.text)
            except AttributeError:
                contract_data[k].append(None)

zakupki_df = pd.concat([zakupki_df, pd.DataFrame(contract_data)]).drop_duplicates('registrationNumber')

100%|██████████| 8181/8181 [03:18<00:00, 41.18it/s]


In [ ]:
len(zakupki_df)

180809

In [ ]:
zakupki_df.to_csv('/content/drive/MyDrive/вкр/zakupki.csv', sep='|', index=None)